In [317]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")
import time
import math
import prettytable
import cvxopt as opt
from cvxopt import blas, solvers
from os import listdir
from os.path import isfile, join
from pathlib import Path

In [318]:
def loadassetfile():
    bond = pd.read_csv("bond.csv", parse_dates=True)
    bond.rename(columns={'bond_ret': 'bond_daily_ret'},inplace=True)
    bond['date'] = pd.to_datetime(bond.date)
    
    equity = pd.read_csv("equity.csv", parse_dates=True)
    equity['equity_daily_ret'] = equity.adj_close.pct_change()
    equity['date'] = pd.to_datetime(equity.date)
    
    forex = pd.read_csv('forex.csv', parse_dates=True)
    forex['forex_daily_ret'] = forex.adj_close.pct_change()
    forex['date'] = pd.to_datetime(forex.date)
    
    gc = pd.read_csv('gc.csv', parse_dates=True)
    gc['gold_daily_ret'] = gc.adj_close.pct_change()
    gc['date'] = pd.to_datetime(gc.date)
    
    return (bond, equity, forex, gc)

In [319]:
def getreturns(bond, equity, forex, gc):
    combined = pd.concat(
        [equity.set_index('date'), bond.set_index('date'), gc.set_index('date'), forex.set_index('date') \
         ], axis=1, join='inner')
    returns = combined.loc[:, ['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']]
    returns.reset_index(inplace=True)
    returns['year'] = returns['date'].dt.year
    returns['month'] = returns['date'].dt.month
    returns['quarter'] = returns.month.apply(lambda v: 'Q1' if v <= 3 else 'Q2' if v <= 6 else 'Q3' if v <= 9 else 'Q4')
    returns['year_quarter'] = returns.year.astype(str) + '-' + returns.quarter.astype(str)
    returns.set_index('date', inplace=True)
    returns = returns.fillna('0')
    return (returns)

In [320]:
def gethistretdaily(returns):
    returns_act = returns.copy()
    returns_act = returns_act.drop(['year', 'month', 'quarter', 'year_quarter'], axis=1)
    returns_act = returns_act.fillna('0')
    daily_ret_his = returns_act.rolling(252).mean()
    daily_ret_his = daily_ret_his.fillna(0)
    return(daily_ret_his)

In [321]:
def readstage1data():
    data = pd.read_csv('SapiatStage1Out.csv', parse_dates=True)
    data['date'] = pd.to_datetime(data.date)
    return data

In [322]:
def get_covar_matrix_hat(x):
    X = np.array([x['equity_var_hat'], x['bond_equity_cov_hat'], x['equity_gold_cov_hat'], x['equity_forex_cov_hat'],
                  x['bond_equity_cov_hat'], x['bond_var_hat'], x['bond_gold_cov_hat'], x['bond_forex_cov_hat'],
                  x['equity_gold_cov_hat'], x['bond_gold_cov_hat'], x['gold_var_hat'], x['forex_gold_cov_hat'],
                  x['equity_forex_cov_hat'], x['bond_forex_cov_hat'], x['forex_gold_cov_hat'], x['forex_var_hat']])
    return opt.matrix(X, (4, 4))

In [323]:
def optimal_portfolio(returns, S):
    n = len(returns)
    N = 100
    mus = [10 ** (5.0 * t / N - 1.0) for t in range(N)]
    # Convert to cvxopt matrices
    pbar = opt.matrix(returns)
    # Create constraint matrices
    G = -opt.matrix(np.eye(n))  # negative n x n identity matrix
    h = opt.matrix(0.0, (n, 1))
    A = opt.matrix(1.0, (1, n))
    b = opt.matrix(1.0)
    # Calculate efficient frontier weights using quadratic programming
    solvers.options['show_progress'] = False
    portfolios = [solvers.qp(mu * S, -pbar, G, h, A, b)['x']
                  for mu in mus]
    ## CALCULATE RISKS AND RETURNS FOR FRONTIER
    returns = []
    returns = [blas.dot(pbar, x) for x in portfolios]
    risks = []
    risks = [np.sqrt(blas.dot(x, S * x)) for x in portfolios]
    risks = [(sum(risks) / len(risks)) if math.isnan(x) else x for x in risks]
    ## CALCULATE THE 2ND DEGREE POLYNOMIAL OF THE FRONTIER CURVE
    m1 = np.polyfit(returns, risks, 2)
    x1 = np.sqrt(abs(m1[2] / m1[0]))
    # CALCULATE THE OPTIMAL PORTFOLIO
    solvers.options['show_progress'] = False
    wt = solvers.qp(opt.matrix(x1 * S), -pbar, G, h, A, b)['x']
    return np.asarray(wt), returns, risks, portfolios

In [324]:
def find_ret(RISK, RETURN, tgt_risk):
    '''
    RISK is a np.array() of risk levels
    RETURN is a np.array() of associated optimum returns
    '''
    assert len(RISK) == len(RETURN)
    last = None
    for (risk, ret) in zip(RISK, RETURN):
        if risk > tgt_risk:
            return last
        last = ret
    return last

In [325]:
def find_weights(RISK, PORTFOLIOS, tgt_risk):
    '''
    RISK is a np.array() of risk levels
    RETURN is a np.array() of associated optimum returns
    Portfolios is weights
    '''
    assert len(RISK) == len(PORTFOLIOS)
    last = None
    for (risk, w) in zip(RISK, PORTFOLIOS):
        if risk > tgt_risk:
            return last
        last = w
    return last

In [326]:
def summary_metrics(R, rf_annual=0.01, show=False):
    pt = prettytable.PrettyTable(['metric', 'value'])
    avg = np.mean(R)
    std_dev = np.std(R)
    rf = rf_annual / 252
    count = len(R)
    sharpe = (avg - rf) / std_dev * np.sqrt(252)
    rpt = {'mean': avg,
           'std_dev': std_dev,
           'Sharpe_ratio': sharpe,
           'count': count}
    for (k, v) in rpt.items():
        pt.add_row([k, v])
    if show:
        print(pt)
    return rpt

In [327]:
def buildweightshat(inputstage1, df_daily_ret_act):
    mean_lookback_returns = []
    quarterly_cov_hat = []
    for x in inputstage1.to_dict('records'):
        sub = df_daily_ret_act[df_daily_ret_act.index < x['date']].tail(252)
        return_vec = sub[['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']].mean().values
        sigma = get_covar_matrix_hat(x)
        mean_lookback_returns += [return_vec]
        quarterly_cov_hat += [sigma]
    risks = []
    weights = []
    for ret, cov in zip(mean_lookback_returns, quarterly_cov_hat):
        w, re, ri, p = optimal_portfolio(ret, cov)
        risks += [ri]
        weights += [p]
    table = {}
    weights_table = []
    for i in range(len(inputstage1)):
        table = {'risks': risks[i], 'weights': weights[i]}
        table = pd.DataFrame(table)
        table = table.sort_values(by=['risks'])
        # low_w = find_weights(table.risks, table.weights, None, 0.01)
        # med_w = find_weights(table.risks, table.weights,0.01, 0.025)
        # high_w = find_weights(table.risks, table.weights, 0.04, None)
        low_w = find_weights(table.risks, table.weights, 0.01)
        med_w = find_weights(table.risks, table.weights, 0.025)
        high_w = find_weights(table.risks, table.weights,0.04)
        g = {'equity_low_w_hat': float(low_w[0]),
             'bond_low_w_hat': float(low_w[1]),
             'gold_low_w_hat': float(low_w[2]),
             'forex_low_w_hat': float(low_w[3]),
             'equity_med_w_hat': float(med_w[0]),
             'bond_med_w_hat': float(med_w[1]),
             'gold_med_w_hat': float(med_w[2]),
             'forex_med_w_hat': float(med_w[3]),
             'equity_high_w_hat': float(high_w[0]),
             'bond_high_w_hat': float(high_w[1]),
             'gold_high_w_hat': float(high_w[2]),
             'forex_high_w_hat': float(high_w[3])}
        weights_table += [g]
    weights_table = pd.DataFrame(weights_table)
    weights_table['calculation_date'] = inputstage1['date']
    weights_table['year'] = weights_table['calculation_date'].dt.year
    weights_table['month'] = weights_table['calculation_date'].dt.month
    weights_table['quarter'] = weights_table.month.apply(
        lambda v: 'Q1' if v <= 3 else 'Q2' if v <= 6 else 'Q3' if v <= 9 else 'Q4')
    weights_table['year_quarter'] = weights_table.year.astype(str) + '-' + weights_table.quarter.astype(str)
    # weights_table = weights_table.drop(['year','month','quarter'], axis = 0)
    weights_table.set_index('year_quarter', inplace=True)
    weights_table = weights_table.fillna('0')
    return weights_table

In [328]:
def buildweightshis(inputstage1, df_daily_ret_act):
    mean_lookback_returns_his = []
    quarterly_cov_hat_his = []
    df_daily_ret_act.reset_index(inplace=True)
    for x in inputstage1.to_dict('records'):
        sub = df_daily_ret_act[df_daily_ret_act['date'] < x['date']].tail(90)
        return_vec = sub[['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']].mean().values
        sub =  sub.drop(['date'], axis=1)
        sub = sub.astype(np.float)
        return_vec_cov = sub.cov()
        sigma = opt.matrix(return_vec_cov.values)
        mean_lookback_returns_his += [return_vec]
        quarterly_cov_hat_his += [sigma]
    risks_his = []
    weights_his = []
    for ret1, cov1 in zip(mean_lookback_returns_his, quarterly_cov_hat_his):
        w, re, ri1, p1 = optimal_portfolio(ret1, cov1)
        risks_his += [ri1]
        weights_his += [p1]
    table = {}
    weights_table_his = []
    for i in range(len(inputstage1)):
        table = {'risks': risks_his[i], 'weights': weights_his[i]}
        table = pd.DataFrame(table)
        table = table.sort_values(by=['risks'])
        # low_w = find_weights(table.risks, table.weights,None, 0.01)
        # med_w = find_weights(table.risks, table.weights,0.01, 0.025)
        # high_w = find_weights(table.risks, table.weights,0.04, None)
        low_w = find_weights(table.risks, table.weights, 0.01)
        med_w = find_weights(table.risks, table.weights, 0.025)
        high_w = find_weights(table.risks, table.weights,0.04)
        g = {'equity_low_w_his': float(low_w[0]),
             'bond_low_w_his': float(low_w[1]),
             'gold_low_w_his': float(low_w[2]),
             'forex_low_w_his': float(low_w[3]),
             'equity_med_w_his': float(med_w[0]),
             'bond_med_w_his': float(med_w[1]),
             'gold_med_w_his': float(med_w[2]),
             'forex_med_w_his': float(med_w[3]),
             'equity_high_w_his': float(high_w[0]),
             'bond_high_w_his': float(high_w[1]),
             'gold_high_w_his': float(high_w[2]),
             'forex_high_w_his': float(high_w[3])}
        weights_table_his += [g]
    weights_table_his = pd.DataFrame(weights_table_his)
    weights_table_his['calculation_date'] = inputstage1['date']
    weights_table_his['year'] = weights_table_his['calculation_date'].dt.year
    weights_table_his['month'] = weights_table_his['calculation_date'].dt.month
    weights_table_his['quarter'] = weights_table_his.month.apply(
        lambda v: 'Q1' if v <= 3 else 'Q2' if v <= 6 else 'Q3' if v <= 9 else 'Q4')
    weights_table_his['year_quarter'] = weights_table_his.year.astype(str) + '-' + weights_table_his.quarter.astype(str)
    weights_table_his.set_index('year_quarter', inplace=True)
    weights_table_his = weights_table_his.fillna('0')
    return weights_table_his

In [352]:
def dailyportfolioreturn(returns_act, inputstage1_act, weights_table, weights_table_his):
#     returns_act.set_index(['date'],inplace=True)
    returns_act = returns_act[inputstage1_act.head(1).index[0]: inputstage1_act.tail(1).index[0]]
    weights_table_his = weights_table_his.drop(['year', 'month', 'quarter', 'calculation_date'], axis=1)
    weights_table = weights_table.drop(['year', 'month', 'quarter', 'calculation_date'], axis=1)

    final_csv_temp = pd.merge(left=returns_act, right=weights_table_his, left_on='year_quarter',
                              right_on='year_quarter')
    final_csv_output = pd.merge(left=final_csv_temp, right=weights_table, left_on='year_quarter',
                                right_on='year_quarter')
    final_csv_output['date'] = returns_act.index
    final_csv_output.set_index('date', inplace=True)
    return_arr = np.array(final_csv_output[['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']].T, dtype=float)
    final_csv_output['ret_low_his'] = np.einsum("ij,ij->j", return_arr
                                                , np.array(
            final_csv_output[['equity_low_w_his', 'bond_low_w_his', 'gold_low_w_his', 'forex_low_w_his']].T,
            dtype=float))
    final_csv_output['ret_med_his'] = np.einsum("ij,ij->j", return_arr
                                                , np.array(
            final_csv_output[['equity_med_w_his', 'bond_med_w_his', 'gold_med_w_his', 'forex_med_w_his']].T,
            dtype=float))
    final_csv_output['ret_high_his'] = np.einsum("ij,ij->j", return_arr
                                                 , np.array(
            final_csv_output[['equity_high_w_his', 'bond_high_w_his', 'gold_high_w_his', 'forex_high_w_his']].T,
            dtype=float))
    final_csv_output['ret_low_hat'] = np.einsum("ij,ij->j", return_arr
                                                , np.array(
            final_csv_output[['equity_low_w_hat', 'bond_low_w_hat', 'gold_low_w_hat', 'forex_low_w_hat']].T,
            dtype=float))
    final_csv_output['ret_med_hat'] = np.einsum("ij,ij->j", return_arr
                                                , np.array(
            final_csv_output[['equity_med_w_hat', 'bond_med_w_hat', 'gold_med_w_hat', 'forex_med_w_hat']].T,
            dtype=float))
    final_csv_output['ret_high_hat'] = np.einsum("ij,ij->j", return_arr
                                                 , np.array(
            final_csv_output[['equity_high_w_hat', 'bond_high_w_hat', 'gold_high_w_hat', 'forex_high_w_hat']].T,
            dtype=float))
    return (final_csv_output)

In [353]:
def savecsvfile(csvstructure, filenamemask):
    if filenamemask.startswith('Stage2OutputReturns'):
        filename = r'outputdata\Stage2\Returns\SapiatStage2Out' + filenamemask + time.strftime("%Y%m%d%H%M%S") + '.csv'
    else:
        filename = r'outputdata\Stage2\Weights\SapiatStage2Out' +  filenamemask + time.strftime("%Y%m%d%H%M%S") + '.csv'
    csvstructure.to_csv(filename)
    print(f' Processing Complete, File :- {filename} has been created\r')

In [331]:
returns

,equity_daily_ret,bond_daily_ret,gold_daily_ret,forex_daily_ret,year,month,quarter,year_quarter
date,,,,,,,,
1982-01-04,0.001550,0,-0.006289,-0.000669,1982,1,Q1,1982-Q1
1982-01-05,-0.021916,-0.000510425,0.021519,0.007551,1982,1,Q1,1982-Q1
1982-01-06,-0.007247,0.000452696,-0.004957,-0.004649,1982,1,Q1,1982-Q1
1982-01-07,-0.002098,0.000322584,-0.012453,0.006100,1982,1,Q1,1982-Q1
1982-01-08,0.005213,0.000838569,0.009584,-0.000758,1982,1,Q1,1982-Q1
...,...,...,...,...,...,...,...,...
2020-04-24,0.013918,-2.2e-05,-0.011749,-0.000498,2020,4,Q2,2020-Q2
2020-04-27,0.014714,1e-05,-0.000525,-0.003387,2020,4,Q2,2020-Q2
2020-04-28,-0.005242,2.83e-05,-0.013644,-0.001699,2020,4,Q2,2020-Q2


In [332]:
df_daily_ret_act

,equity_daily_ret,bond_daily_ret,gold_daily_ret,forex_daily_ret
date,,,,
1982-01-04,0.001550,0,-0.006289,-0.000669
1982-01-05,-0.021916,-0.000510425,0.021519,0.007551
1982-01-06,-0.007247,0.000452696,-0.004957,-0.004649
1982-01-07,-0.002098,0.000322584,-0.012453,0.006100
1982-01-08,0.005213,0.000838569,0.009584,-0.000758
...,...,...,...,...
2020-04-24,0.013918,-2.2e-05,-0.011749,-0.000498
2020-04-27,0.014714,1e-05,-0.000525,-0.003387
2020-04-28,-0.005242,2.83e-05,-0.013644,-0.001699


In [354]:
    bond, equity, forex, gc = loadassetfile()
    inputstage1 = readstage1data()
    inputstage1_act = inputstage1.copy()
    inputstage1_act.set_index('date', inplace=True)
    returns = getreturns(bond, equity, forex, gc)
    df_daily_ret_act = returns.copy()
    df_daily_ret_act = df_daily_ret_act.drop(['year', 'month', 'quarter','year_quarter'], axis=1)
#     df_daily_ret_act.fillna('0')
    returns_act = returns.copy()
#     returns_act = df_daily_ret_act.drop(['year', 'month', 'quarter','year_quarter'], axis=1)
#     df_daily_ret_act.fillna('0')
#     df_daily_ret_act = returns[['equity_ret', 'bond_ret', 'gc_ret', 'forex_ret']]
#     df_daily_ret_his = returns[['equity_ret', 'bond_ret', 'gc_ret', 'forex_ret']].rolling(252).mean()
#     df_daily_ret_his['date'] = returns.index
#     df_daily_ret_his = df_daily_ret_his.fillna('0')
#     inputstage1['date'] = pd.to_datetime(inputstage1.date)
    weights_table_hat = buildweightshat(inputstage1, df_daily_ret_act)
    weights_table_his = buildweightshis(inputstage1, df_daily_ret_act)
    dailyportfolioreturncsv = dailyportfolioreturn(returns_act, inputstage1_act, weights_table_hat, weights_table_his)
#     savecsvfile(dailyportfolioreturncsv, 'Stage2OutputReturns')
#     savecsvfile(weights_table_hat, 'Stage2OutputWeightsHat')
#     savecsvfile(weights_table_his, 'Stage2OutputWeightsHis')

In [355]:
dailyportfolioreturncsv.to_csv('Stage_2_out_Abir.csv')

In [350]:
dailyportfolioreturncsv = dailyportfolioreturn(returns_act, inputstage1_act, weights_table_hat, weights_table_his)

KeyError: "None of [Index(['equity_ret', 'bond_ret', 'gc_ret', 'forex_ret'], dtype='object')] are in the [columns]"

In [280]:
    mean_lookback_returns_his = []
    quarterly_cov_hat_his = []
    for x in inputstage1.to_dict('records'):
        sub = df_daily_ret_act[df_daily_ret_act.index < x['date']].tail(252)
        sub = sub.fillna('0')
        return_vec = sub[['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']].mean().values
        sub.reset_index(inplace=True)
        sub =  sub.drop(['date'], axis=1)
        sub = sub.fillna('0')
        return_vec_cov = sub[['bond_daily_ret','equity_daily_ret',  'gold_daily_ret',  'forex_daily_ret']].cov()
#         return_vec_cov = np.cov(sub[['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']].astype(object))
        print(return_vec_cov.shape)
#         sigma = opt.matrix(return_vec_cov.values)
#         mean_lookback_returns_his += [return_vec]
#         quarterly_cov_hat_his += [sigma]
        break

TypeError: '<' not supported between instances of 'int' and 'Timestamp'

In [272]:
df_daily_ret_act.isna().sum()

equity_daily_ret    0
bond_daily_ret      0
gold_daily_ret      0
forex_daily_ret     0
dtype: int64

In [147]:
    mean_lookback_returns = []
    quarterly_cov_hat = []
    for x in inputstage1.to_dict('records'):
        sub = df_daily_ret_act[df_daily_ret_act.index < x['date']].tail(252)
        return_vec = sub[['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']].mean().values
        sigma = get_covar_matrix_hat(x)
        print(sigma)
        break

[ 1.89e-05  6.32e-08  2.69e-06 -1.03e-06]
[ 6.32e-08  8.41e-09 -5.87e-08  4.83e-09]
[ 2.69e-06 -5.87e-08  7.88e-05 -1.75e-05]
[-1.03e-06  4.83e-09 -1.75e-05  2.11e-05]



In [303]:
    for x in inputstage1.to_dict('records'):
        sub = df_daily_ret_act[df_daily_ret_act['date'] < x['date']].tail(90)
        return_vec = sub[['equity_daily_ret', 'bond_daily_ret', 'gold_daily_ret', 'forex_daily_ret']].mean().values
#         sub.reset_index(inplace=True)
        sub =  sub.drop(['date'], axis=1)
        sub = sub.astype(np.double)
#         subsub.bond_daily_ret = sub.bond_daily_ret.fillna(sub.bond_daily_ret.mean())
        print(sub)
#         print(sub.isna().sum())
        return_vec_cov1 = sub.cov()
#         if (len(return_vec_cov.columns)) == 3:
#             print(x['date'])
        print(return_vec_cov1)
        break

      equity_daily_ret  bond_daily_ret  gold_daily_ret  forex_daily_ret
6457         -0.007069        0.000325        0.011439        -0.000922
6458         -0.013212        0.000373       -0.023970         0.003430
6459          0.005230       -0.000160       -0.005290        -0.003155
6460         -0.017460        0.000286       -0.013801        -0.000659
6461          0.004486        0.000473        0.021312        -0.007522
...                ...             ...             ...              ...
6542          0.015321       -0.001457        0.000000         0.002749
6543          0.002304       -0.000091        0.001080        -0.009184
6544         -0.008766        0.000036        0.021580        -0.010653
6545         -0.011460        0.000011        0.000000         0.002098
6546         -0.007950       -0.000039       -0.013202         0.000279

[90 rows x 4 columns]
                  equity_daily_ret  bond_daily_ret  gold_daily_ret  \
equity_daily_ret          0.000213   -1.266